In [ ]:
import json

import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Raw Data Exploration

In [ ]:
def parse_datetime(datetime_column):
    return pd.to_datetime(datetime_column, unit = 's')

In [ ]:
# Read raw messages, constructing a MultiIndex from device_id and correctly parsed datetime
raw_data = pd.read_csv('data/raw_messages.csv', 
                       index_col=['device_id','datetime'], 
                       parse_dates=['datetime'], 
                       date_parser=parse_datetime).sort_index()

# Drop duplicates.
raw_data.drop_duplicates(inplace=True)
# Remove non-alphanumeric noisy characters from raw_message column.
raw_data['raw_message'] = raw_data['raw_message'].str.replace('[^a-zA-Z0-9,.]','', regex=True)
# Expand raw message column into separate columns.
raw_data = pd.concat([raw_data.drop('raw_message', axis=1), raw_data['raw_message'].str.split(',', expand=True)], 
                     axis=1)
# Rename the expanded columns into correct raw message attributes.
rename_dict = {0: 'status', 
               1: 'lat', 
               2: 'lat_dir', 
               3: 'lon', 
               4: 'lon_dir', 
               5: 'spd_over_grnd', 
               6: 'true_course', 
               7: 'datestamp', 
               8: 'mag_variation', 
               9: 'mag_var_dir'}
raw_data.rename(columns=rename_dict, inplace=True)

# Fix dtypes of numeric columns.
dtypes_dict = {'lat': np.float64,
               'lon': np.float64,
               'spd_over_grnd': np.float64,
               'true_course': np.float64,
               'datestamp': np.int64,
               'mag_variation': np.float64}
raw_data = raw_data.astype(dtypes_dict, errors='raise').round(decimals=3) 

In [ ]:
# Print overview of raw data for st-1a2090.
raw_data.xs('st-1a2090',level='device_id').head()

In [ ]:
clean_raw_data = pd.read_csv('data/raw_messages_clean.csv', 
                       index_col=['device_id','datetime'], 
                       parse_dates=['datetime']).sort_index().round(decimals=3)

In [ ]:
# Print overview of clean raw data for st-1a2090.
clean_raw_data.xs('st-1a2090',level='device_id').head()

In [ ]:
# Show that after rounding floats to 3 decimals, raw_data and clean_raw_data are identical.
pd.concat([raw_data,clean_raw_data]).drop_duplicates(keep=False).shape[0]

# Weather Data Exploration

In [ ]:
# Read weather_data.json at weather stations level (without normalizing data field).
weather_stations = pd.read_json('data/weather_data.json').rename(columns={'lat': 's_lat', 
                                                                          'lon': 's_lon'}).round(decimals=3)


In [ ]:
# Print overview of weather stations.
weather_stations

In [ ]:
# Read full weather data by normalizing data field into separate columns.
with open('data/weather_data.json') as f:
  weather_data_json = json.load(f)

weather_data= pd.json_normalize(weather_data_json,record_path='data', 
                                meta=['station_id','lat','lon']).rename(columns={'lat': 's_lat', 
                                                                                 'lon': 's_lon',
                                                                                 'datetime': 's_datetime'})
weather_data['timestamp_utc'] = pd.to_datetime(weather_data['timestamp_utc'])
weather_data = weather_data.set_index('timestamp_utc').sort_index()
dtypes_dict = {'s_lat': np.float64,
               's_lon': np.float64}
weather_data = weather_data.astype(dtypes_dict, errors='raise').round(decimals=3) 

In [ ]:
# Print overview of full weather data.
weather_data.head()

In [ ]:
# Steal haversine formula implementation from internet. 
# https://medium.com/analytics-vidhya/finding-nearest-pair-of-latitude-and-longitude-match-using-python
from math import radians, cos, sin, asin, sqrt
def dist(lat1, long1, lat2, long2):
    """
Replicating the same formula as mentioned in Wiki
    """
    # convert decimal degrees to radians 
    lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
    # haversine formula 
    dlon = long2 - long1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [ ]:
# Function for applying on raw data to find nearest weather station for each data point.
def find_nearest_weather_station(lat, long):
    distances = weather_stations.apply(lambda row: dist(lat, long, row['s_lat'], row['s_lon']), 
                                       axis=1)
    return weather_stations.loc[distances.idxmin(), ['s_lat', 's_lon']]

In [ ]:
# Find nearest weather station for each data point and add its coordinates to raw data.
raw_data[['s_lat', 's_lon']] = raw_data.apply(lambda row: find_nearest_weather_station(row['lat'], row['lon']), 
                                              axis=1)

In [ ]:
# Merge asof on datetime indexes of raw data and weather data (nearest).
# Merge by exact match of station lat and lon, so that data of the nearest station is grabbed.
# Result will be a dataframe with weather data columns added on raw data.
date = pd.to_datetime('2019-02-13')
device_id = 'st-1a2090'
raw_and_weather = pd.merge_asof(raw_data.xs(device_id,level='device_id').loc[date:date+pd.DateOffset(1)],
                                weather_data,
                                left_index = True, 
                                right_index = True, by=['s_lat','s_lon'], 
                                direction='nearest', 
                                tolerance=pd.Timedelta('1H'))
raw_and_weather = pd.concat({device_id: raw_and_weather}, names=['device_id'])

In [ ]:
raw_and_weather

# Metric a

In [ ]:
# For how many ships do we have available data?
raw_data.index.get_level_values('device_id').unique().to_list()

# Metric b

In [ ]:
# Avg speed for all available ships for each hour of the date 2019-02-13.
date = pd.to_datetime('2019-02-13')
raw_data.loc[pd.IndexSlice[:, date:date+pd.DateOffset(1)], :]['spd_over_grnd'].groupby([pd.Grouper(level='device_id'),
                                                                                        pd.Grouper(level='datetime',
                                                                                                   freq='1H')]).mean().to_frame()

# Metric c

In [ ]:
# Max & min wind speed for every available day for ship ”st-1a2090” only.
date = pd.to_datetime('2019-02-13')
device_id = 'st-1a2090'
raw_and_weather.loc[pd.IndexSlice[device_id, date:date+pd.DateOffset(1)], :]['wind_spd'].groupby([pd.Grouper(level='device_id'),
                                                                                                  pd.Grouper(level='datetime',
                                                                                                             freq='1H')]).max().to_frame()

In [ ]:
raw_and_weather.loc[pd.IndexSlice[device_id, date:date+pd.DateOffset(1)], :]['wind_spd'].groupby([pd.Grouper(level='device_id'),
                                                                                                  pd.Grouper(level='datetime',
                                                                                                             freq='1H')]).min().to_frame()

# Metric d

In [ ]:
# A way to visualize full weather conditions (example fields: general description, temperature, wind speed) 
# across route of the ship ”st-1a2090” for date 2019-02-13.
date = pd.to_datetime('2019-02-13')
device_id = 'st-1a2090'
plot_data = raw_and_weather.loc[pd.IndexSlice[device_id, date:date+pd.DateOffset(1)], :]
plot_data['weather_text'] = 'Timestamp: ' + plot_data.index.get_level_values(level='datetime').astype(str) + ', ' + 'Description: ' + plot_data['weather.description'].astype(str) + ', ' + 'Temperature: '+ plot_data['temp'].astype(str) + ', '+ 'Wind Speed: ' + plot_data['wind_spd'].astype(str)
plot_data.reset_index(drop=True,inplace=True)

fig = go.Figure(data=go.Scattergeo(lon = plot_data['lon'],
                                   lat = plot_data['lat'],
                                   text = plot_data['weather_text'], 
                                   mode = 'lines', 
                                   line = dict(color = 'red')))

fig.update_geos(fitbounds='locations',
                resolution=50,
                showocean=True, oceancolor="LightBlue",
                showlakes=True, lakecolor="Blue",
                showrivers=True, rivercolor="Blue")
fig.update_layout(title = f"Route for {device_id} on {date}" + '<br>(Hover for weather conditions)', 
                  geo_scope='europe')            
fig.show()